### 절대 모멘텀
1. 파생변수 'STD-YM' 생성 -> 인덱스에서 년-월을 추출하여 대입 
2. 'STD-YM' 별 마지막날의 데이터들을 month_last_df 추가 (단순 행 결합)
3. 전월의 수정 종가 값을 가진 파생변수 생성 
4. 전년도의 수정 종가 값을 가진 파생변수 생성 
5. 전월의 데이터와 전년도의 데이터를 이용하여 거래 내역 생성 
6. 수익율 계산

In [ ]:
import pandas as pd
import numpy as np 
from datetime import datetime
import warnings

In [ ]:
# 위험 메시지 출력 필터
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("../../csv/AMZN.csv")

In [ ]:
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

In [ ]:
# index를 시계열로 전환
df.index = pd.to_datetime(df.index)

In [ ]:
# df에 STD-YM 컬럼을 생성하여 인덱스에서 년도-월 데이터를 추출하여 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [ ]:
# 빈 데이터프레임에 월말의 데이터들을 행결합 
# STD-YM에서 다음 행과 현재 행이 다르다면
flag = df['STD-YM'] != df['STD-YM'].shift(-1)
df.loc[flag, ]

In [ ]:
# 기준년월의 유니크 값을 따로 생성 
month_list = df['STD-YM'].unique()
month_list

In [ ]:
month_last_df = pd.DataFrame()

for month in month_list:
    # flag = df['STD-YM'] == month
    # data = df.loc[flag, ].tail(1)
    data = df.loc[month, ].tail(1)
    month_last_df = pd.concat(
        [month_last_df, data], axis=0
    )

In [ ]:
month_last_df = month_last_df[['Adj Close', 'STD-YM']]
month_last_df.head()

In [ ]:
# 전월의 수정종가 파생변수(BF-1M) 생성
month_last_df['BF-1M'] = month_last_df['Adj Close'].shift(1).fillna(0)
# 전년도의 수정종가 파생변수(BF-12M) 생성
month_last_df['BF-12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

In [ ]:
month_last_df

- 거래 내역 추가 
    - (전월의 수정주가) / (전년도의 수정주가) - 1 값이 0보다 크고 무한대가 아닌 경우가 매수 타이밍
    - 모멘텀 인덱스가 위의 조건에 부합할 경우 df의 trade 컬럼에 해당 월에 구매 내역 추가 

In [ ]:
df['trade'] = ""

# month_last_df의 인덱스를 기준으로 반복문 실행 
for idx in month_last_df.index:
    signal = ""

    # 절대 모멘텀 인덱스를 계산 
    momentum_index = (month_last_df.loc[idx, 'BF-1M'] / 
                    month_last_df.loc[idx, 'BF-12M']) - 1
    # print(momentum_index)
    # break
    # 조건식 : 모멘텀 인덱스가 0보다 크고 무한대가 아닌
    # 참인 경우 데이터 if 조건식 else 거짓인 경우 데이터
    flag = True if (momentum_index > 0) & (momentum_index != np.inf) \
        else False

    if flag:
        signal = 'buy'
    print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}, signal : {signal}")
    df.loc[idx:, 'trade'] = signal

In [ ]:
df['trade'].value_counts()

In [ ]:
# 수익율 계산
# 매수 일자 : 전날의 trade가 ""이고 오늘의 trade "buy"
# 매도 일자 : 전날의 trade가 "buy"이고 오늘의 trade ""

df['rtn'] = 1

for idx in df.index:
    # 매수의 조건식 
    if (df.shift().loc[idx, 'trade'] == "") & \
        (df.loc[idx, 'trade'] == "buy"):
        buy = df.loc[idx, 'Adj Close']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도의 조건식
    elif (df.shift().loc[idx, 'trade'] == 'buy') & \
        (df.loc[idx, 'trade'] == ""):
        sell = df.loc[idx, 'Adj Close']
        rtn = sell / buy
        df.loc[idx, 'rtn'] = rtn
        print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")

In [ ]:
df['acc_rtn'] = df['rtn'].cumprod()

In [ ]:
acc_rtn = 1
for i in df.index:
    rtn = df.loc[i, 'rtn']
    acc_rtn *= rtn
acc_rtn

In [ ]:
df.iloc[-1, ]['Adj Close'] / df.iloc[0, ]['Adj Close']

In [ ]:
df2 = df.drop(
    ['trade', 'rtn', 'acc_rtn'], 
    axis = 1
)

In [ ]:
df2['trade'] = ""

# month_last_df의 인덱스를 기준으로 반복문 실행 
for idx in month_last_df.index:
    # 절대 모멘텀 인덱스를 계산 
    momentum_index = (month_last_df.loc[idx, 'BF-1M'] / 
                    month_last_df.loc[idx, 'BF-12M']) - 1

    flag = True if (momentum_index > 0) & (momentum_index != np.inf) \
        else False

    if flag:
        # # 모멘텀 인덱스가 구매 신호 주면 해당 월말부터 
        # # 마지막 데이터까지 buy 채워준다.
        # df2.loc[idx:, 'trade'] = 'buy'
        # 모멘텀 인덱스가 구매 신호를 주면 해당 월말 다음 행의 STD-YM 추출하여
        # 해당 조건 맞는 인덱스를 필터링 하여 trade에 buy을 대입 
        std_ym = df2.shift(-1).loc[idx, 'STD-YM']
        df2.loc[std_ym, 'trade'] = 'buy'
        print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}")
    

In [ ]:
df2['trade'].value_counts()

In [ ]:
# 수익율 계산
# 매수 일자 : 전날의 trade가 ""이고 오늘의 trade "buy"
# 매도 일자 : 전날의 trade가 "buy"이고 오늘의 trade ""

df2['rtn'] = 1

for idx in df2.index:
    # 매수의 조건식 
    if (df2.shift().loc[idx, 'trade'] == "") & \
        (df2.loc[idx, 'trade'] == "buy"):
        buy = df2.loc[idx, 'Adj Close']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도의 조건식
    elif (df2.shift().loc[idx, 'trade'] == 'buy') & \
        (df2.loc[idx, 'trade'] == ""):
        sell = df2.loc[idx, 'Adj Close']
        rtn = sell / buy
        df2.loc[idx, 'rtn'] = rtn
        print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")

In [ ]:
df2['rtn'].cumprod()